# Day 20 - o1-mini (GHC)

In [1]:
from collections import deque

def read_map(filename):
    with open(filename, 'r') as f:
        grid = [list(line.strip()) for line in f]
    return grid

def find_positions(grid):
    start = end = None
    for y, row in enumerate(grid):
        for x, val in enumerate(row):
            if val == 'S':
                start = (x, y)
            elif val == 'E':
                end = (x, y)
    return start, end

def bfs(grid, start, end, cheat=False):
    queue = deque()
    visited = {}
    queue.append((start[0], start[1], 0, 0))  # x, y, steps, cheats_used
    visited[(start[0], start[1], 0)] = 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    while queue:
        x, y, steps, cheats = queue.popleft()
        if (x, y) == end:
            return steps
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0<=ny<len(grid) and 0<=nx<len(grid[0]):
                cell = grid[ny][nx]
                if cell != '#':
                    if (nx, ny, cheats) not in visited:
                        visited[(nx, ny, cheats)] = steps + 1
                        queue.append((nx, ny, steps + 1, cheats))
                elif cheat and cheats == 0:
                    if (nx, ny, cheats + 1) not in visited:
                        visited[(nx, ny, cheats + 1)] = steps + 1
                        queue.append((nx, ny, steps + 1, cheats + 1))
    return float('inf')

def main():
    grid = read_map('input.txt')
    start, end = find_positions(grid)
    base_time = bfs(grid, start, end)
    cheats = []
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == '#':
                grid[y][x] = '.'  # Temporarily remove wall
                new_time = bfs(grid, start, end)
                if new_time < base_time:
                    savings = base_time - new_time
                    cheats.append(savings)
                grid[y][x] = '#'  # Restore wall
    count = sum(1 for s in cheats if s >= 100)
    print(count)

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [2]:
from collections import deque

def read_map(filename):
    with open(filename, 'r') as f:
        grid = [list(line.strip()) for line in f]
    return grid

def find_positions(grid):
    start = end = None
    for y, row in enumerate(grid):
        for x, val in enumerate(row):
            if val == 'S':
                start = (x, y)
            elif val == 'E':
                end = (x, y)
    return start, end

def bfs(grid, start, end):
    queue = deque()
    visited = {}
    queue.append((start[0], start[1], 0, False, 0))  # x, y, steps, cheating, cheat_steps
    visited[(start[0], start[1], False, 0)] = 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    cheats = {}
    while queue:
        x, y, steps, cheating, cheat_steps = queue.popleft()
        if (x, y) == end and not cheating:
            continue
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(grid) and 0 <= nx < len(grid[0]):
                cell = grid[ny][nx]
                can_move = cell != '#'
                if can_move or cheating:
                    new_cheating = cheating
                    new_cheat_steps = cheat_steps
                    if not can_move and not cheating:
                        new_cheating = True
                        new_cheat_steps = 1
                    elif cheating:
                        new_cheat_steps += 1
                        if new_cheat_steps > 2:
                            continue
                        if new_cheat_steps == 2:
                            new_cheating = False
                    state = (nx, ny, new_cheating, new_cheat_steps)
                    if state not in visited or visited[state] > steps + 1:
                        visited[state] = steps + 1
                        queue.append((nx, ny, steps + 1, new_cheating, new_cheat_steps))
                        if (x, y, nx, ny) not in cheats and new_cheating:
                            cheats[(x, y, nx, ny)] = steps + 1
    # Calculate savings
    base_time = bfs_no_cheat(grid, start, end)
    count = 0
    for cheat_pos, cheat_time in cheats.items():
        savings = base_time - cheat_time
        if savings >= 100:
            count += 1
    print(count)

def bfs_no_cheat(grid, start, end):
    queue = deque()
    visited = set()
    queue.append((start[0], start[1], 0))
    visited.add((start[0], start[1]))
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    while queue:
        x, y, steps = queue.popleft()
        if (x, y) == end:
            return steps
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(grid) and 0 <= nx < len(grid[0]):
                if grid[ny][nx] != '#' and (nx, ny) not in visited:
                    visited.add((nx, ny))
                    queue.append((nx, ny, steps + 1))
    return float('inf')

def main():
    grid = read_map('input.txt')
    start, end = find_positions(grid)
    bfs(grid, start, end)

if __name__ == "__main__":
    main()

0


In [3]:
from collections import deque

def read_map(filename):
    with open(filename, 'r') as f:
        grid = [list(line.strip()) for line in f]
    return grid

def find_positions(grid):
    start = end = None
    for y, row in enumerate(grid):
        for x, val in enumerate(row):
            if val == 'S':
                start = (x, y)
            elif val == 'E':
                end = (x, y)
    return start, end

def bfs(grid, start, end, cheat_start=None, cheat_length=0):
    queue = deque()
    visited = {}
    queue.append((start[0], start[1], 0, False, 0))  # x, y, steps, cheating, cheat_steps
    visited[(start[0], start[1], False, 0)] = 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    while queue:
        x, y, steps, cheating, cheat_steps = queue.popleft()
        if (x, y) == end and not cheating:
            return steps
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(grid) and 0 <= nx < len(grid[0]):
                cell = grid[ny][nx]
                can_move = cell != '#'
                new_cheating = cheating
                new_cheat_steps = cheat_steps
                if can_move:
                    if cheating:
                        new_cheat_steps += 1
                        if new_cheat_steps > cheat_length:
                            continue
                        if new_cheat_steps == cheat_length:
                            new_cheating = False
                else:
                    if not cheating and cheat_length > 0:
                        new_cheating = True
                        new_cheat_steps = 1
                    else:
                        continue
                state = (nx, ny, new_cheating, new_cheat_steps)
                if state not in visited or visited[state] > steps + 1:
                    visited[state] = steps + 1
                    queue.append((nx, ny, steps + 1, new_cheating, new_cheat_steps))
    return float('inf')

def main():
    grid = read_map('input.txt')
    start, end = find_positions(grid)
    base_time = bfs(grid, start, end)
    cheats = set()
    
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            if grid[y][x] == 'S' or grid[y][x] == '.' :
                # Try cheating for 1 step
                time_with_cheat = bfs(grid, start, end, cheat_start=(x, y), cheat_length=1)
                savings = base_time - time_with_cheat
                if savings >= 100:
                    cheats.add((x, y, '1'))
                
                # Try cheating for 2 steps
                time_with_cheat = bfs(grid, start, end, cheat_start=(x, y), cheat_length=2)
                savings = base_time - time_with_cheat
                if savings >= 100:
                    cheats.add((x, y, '2'))
    
    print(len(cheats))

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [4]:
from collections import deque

def read_map(filename):
    with open(filename, 'r') as f:
        grid = [list(line.strip()) for line in f]
    return grid

def find_positions(grid):
    start = end = None
    for y, row in enumerate(grid):
        for x, val in enumerate(row):
            if val == 'S':
                start = (x, y)
            elif val == 'E':
                end = (x, y)
    return start, end

def bfs_with_cheat(grid, start, end):
    queue = deque()
    # State: (x, y, cheat_used, cheat_steps_remaining)
    queue.append((start[0], start[1], False, 0, 0))  # x, y, cheat_used, cheat_steps_remaining, steps
    visited = {}
    visited[(start[0], start[1], False, 0)] = 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    cheats = {}

    while queue:
        x, y, cheat_used, cheat_steps, steps = queue.popleft()
        if (x, y) == end and not cheat_used:
            base_time = steps
            continue
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(grid) and 0 <= nx < len(grid[0]):
                cell = grid[ny][nx]
                can_move = cell != '#'
                new_cheat_used = cheat_used
                new_cheat_steps = cheat_steps
                if can_move:
                    if cheat_steps > 0:
                        new_cheat_steps -= 1
                else:
                    if not cheat_used and cheat_steps == 0:
                        new_cheat_used = True
                        new_cheat_steps = 1  # Allow passing through wall for 1 step
                    else:
                        continue
                state = (nx, ny, new_cheat_used, new_cheat_steps)
                if state not in visited or visited[state] > steps + 1:
                    visited[state] = steps + 1
                    queue.append((nx, ny, new_cheat_used, new_cheat_steps, steps + 1))
                    if new_cheat_used and (x, y) not in cheats:
                        cheats[(x, y)] = steps + 1
    return base_time, cheats

def main():
    grid = read_map('input.txt')
    start, end = find_positions(grid)
    base_time, cheats = bfs_with_cheat(grid, start, end)
    count = 0
    for cheat_start, cheat_time in cheats.items():
        savings = base_time - cheat_time
        if savings >= 100:
            count += 1
    print(count)

if __name__ == "__main__":
        main()

9503


In [5]:
from collections import deque

def read_map(filename):
    with open(filename, 'r') as f:
        grid = [list(line.strip()) for line in f]
    return grid

def find_positions(grid):
    start = end = None
    for y, row in enumerate(grid):
        for x, val in enumerate(row):
            if val == 'S':
                start = (x, y)
            elif val == 'E':
                end = (x, y)
    return start, end

def bfs(grid, start, end):
    queue = deque()
    # State: (x, y, cheat_used, cheat_steps_remaining)
    queue.append((start[0], start[1], False, 0, 0))  # x, y, cheat_used, cheat_steps_remaining, steps
    visited = {}
    visited[(start[0], start[1], False, 0)] = 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    
    base_time = float('inf')
    cheats_info = {}

    while queue:
        x, y, cheat_used, cheat_steps, steps = queue.popleft()
        
        if (x, y) == end and not cheat_used:
            if steps < base_time:
                base_time = steps
            continue

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(grid) and 0 <= nx < len(grid[0]):
                cell = grid[ny][nx]
                can_move = cell != '#'
                
                new_cheat_used = cheat_used
                new_cheat_steps = cheat_steps
                cheat_activated = False
                cheat_start = None
                cheat_end = None

                if can_move:
                    if cheat_steps > 0:
                        new_cheat_steps -= 1
                        if new_cheat_steps == 0:
                            new_cheat_used = False
                else:
                    if not cheat_used and cheat_steps == 0:
                        # Activate cheat
                        new_cheat_used = True
                        new_cheat_steps = 2
                        cheat_activated = True
                        cheat_start = (x, y)
                    else:
                        continue  # Cannot move into wall without cheat

                state = (nx, ny, new_cheat_used, new_cheat_steps)
                if state not in visited or visited[state] > steps + 1:
                    visited[state] = steps + 1
                    queue.append((nx, ny, new_cheat_used, new_cheat_steps, steps + 1))
                    
                    if cheat_activated:
                        # Record the cheat start
                        cheats_info[(cheat_start)] = []
                    
                    if cheat_used and cheat_steps > 0:
                        # Record cheat end when cheat_steps reaches 0
                        cheats_info.setdefault((cheat_start), []).append((nx, ny))
    
    return base_time, cheats_info

def bfs_with_cheat(grid, start, end, cheat_start, cheat_end):
    queue = deque()
    # State: (x, y, cheat_used, cheat_steps_remaining)
    queue.append((start[0], start[1], False, 0, 0))  # x, y, cheat_used, cheat_steps_remaining, steps
    visited = {}
    visited[(start[0], start[1], False, 0)] = 0
    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    
    while queue:
        x, y, cheat_used, cheat_steps, steps = queue.popleft()
        
        if (x, y) == end and not cheat_used:
            return steps

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= ny < len(grid) and 0 <= nx < len(grid[0]):
                cell = grid[ny][nx]
                can_move = cell != '#'
                
                new_cheat_used = cheat_used
                new_cheat_steps = cheat_steps
                cheat_triggered = False

                if can_move:
                    if cheat_steps > 0:
                        new_cheat_steps -= 1
                        if new_cheat_steps == 0:
                            new_cheat_used = False
                else:
                    if not cheat_used and (x, y) == cheat_start:
                        # Activate cheat
                        new_cheat_used = True
                        new_cheat_steps = 2
                        cheat_triggered = True
                    else:
                        continue  # Cannot move into wall without cheat

                state = (nx, ny, new_cheat_used, new_cheat_steps)
                if state not in visited or visited[state] > steps + 1:
                    visited[state] = steps + 1
                    queue.append((nx, ny, new_cheat_used, new_cheat_steps, steps + 1))
    
    return float('inf')

def main():
    grid = read_map('input.txt')
    start, end = find_positions(grid)
    if not start or not end:
        print("Start or End position not found.")
        return

    base_time, cheats_info = bfs(grid, start, end)
    if base_time == float('inf'):
        print("No path from Start to End.")
        return

    cheat_saves = {}

    for cheat_start, cheat_ends in cheats_info.items():
        if not cheat_ends:
            continue
        for cheat_end in cheat_ends:
            # Calculate path with this cheat
            path_time = bfs_with_cheat(grid, start, end, cheat_start, cheat_end)
            if path_time < base_time:
                saving = base_time - path_time
                if saving >= 100:
                    # Use tuple of start and end positions as unique identifier
                    cheat_saves[(cheat_start, cheat_end)] = saving

    count = len(cheat_saves)
    print(count)

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

## Part 2